# Introduction to pyEELSMODEL
In this notebook, a small overview is given on the different functionalities of pyEELSMODEL. The other notebooks are more in-depth on how to use the model-based approach for elemental quantification. 

In [1]:
%matplotlib qt
#important for interactive plotting tools

In [42]:
import pyEELSMODEL.api as em
import os
import numpy as np
import matplotlib.pyplot as plt

## Loading and Saving Data
Different file formats such as *.msa*, *.dm3* and *.dm4* are supported. The Spectrum and MultiSpectrum can be saved as a *.hdf5* file. \
For the *.dm3/4* files there could be a bug in the loader when the energy axis has negative values (zero loss peak). If this is bug arises then it can be fixed to put flip_sign = True in the loading function. 

### Spectrum loading
Many function contain docstrings, one can access the documentation via following command

In [ ]:
em.Spectrum.load?

In [ ]:
filename = r'.\data\hl.msa'
s = em.Spectrum.load(filename)

In [ ]:
s.plot()

A spectrum can also be created by first creating a SpectrumShape which needs to know the dispersion, offset and size. Next, the Spectrum needs to have the Spectrumshape and raw data

In [ ]:
dispersion = 1
offset = 100
size = 100
data = np.random.random(size)

specshape = em.Spectrumshape(dispersion, offset, size)
s = em.Spectrum(specshape, data)

In [ ]:
s.plot()

When the energy axis is given and the raw data is available, the following code can be used to create a spectrum

In [ ]:
energy_axis = dispersion*np.arange(size)+offset
data = np.random.random(size)
s = em.Spectrum.from_numpy(data, energy_axis)

In [ ]:
s.plot()

#### Spectrum saving
The spectrum can be saved as *.hdf5*. 

savename = r'.\data\hl_test.hdf5'
s.save_hdf5(savename)

In [ ]:
p = em.Spectrum.load(savename)

In [ ]:
p.plot()

#### MultiSpectrum loading

In [ ]:
filename = r'.\data\multi.hdf5'
s = em.MultiSpectrum.load(filename)

In [ ]:
#shows the current selected spectrum
print('current spectrum id is: ' + str(s.currentspectrumid))
s.plot()

In [ ]:
s.setcurrentspectrum((2,1))

In [ ]:
print('current spectrum id is: ' + str(s.currentspectrumid))
s.plot()

In [ ]:
#plots the average spectrum
s.mean().plot()

A multispectrum can also be created by having a MultiSpectrumshape object together with the raw data

In [ ]:
dispersion = 1
offset = 100
size = 100
xsize = 4
ysize = 2
data = np.random.random((xsize, ysize, size))

m_specshape = em.MultiSpectrumshape(dispersion, offset, size, xsize, ysize)
s = em.MultiSpectrum(m_specshape, data)

Or from a numpy array and energy axis

In [ ]:
energy_axis = dispersion*np.arange(size)+offset
data = np.random.random((xsize, ysize, size))
s = em.MultiSpectrum.from_numpy(data, energy_axis)

#### MultiSpectrum saving


In [ ]:
s.save_hdf5?

In [ ]:
savename = r'.\data\multi_test.hdf5'
s.save_hdf5(savename)

## STEM-EELS Simulation
In this part, a simulated STEM-EELS map is created which will be used to showcase the different functionalities of pyEELSMODEL.
It contains a low-loss and core-loss. The core-loss has carbon K edge, nitrogen K edge, oxygen K edge and iron L edge and has a scan size of 128x128. *In the CoreLossExample notebook, the procedure of simulating this multispectrum is shown.* 

In [3]:
from pyEELSMODEL.misc.data_simulator import simulate_data

In [4]:
hl, ll = simulate_data()

16384it [00:13, 1192.59it/s]
16384it [00:39, 417.68it/s]
16384it [00:00, 34259.65it/s]


Multispectrum is simulated


## Aligning multispectra
The core-loss gets aligned using the low-loss. Multiple methods are available for find the appropriate shifts and correct for it.

1. **FastAlignZeroLoss**: Uses the energy at which the maximum intensity is measured. The found shifts are applied by rolling each spectra to make it align. This method is fast and does not modify the inputted data via interpolation. Hence it cannot find subpixel positions. This method works best when the zero-loss peak is sharp and has a high intensity which is valid in most cases. In our experience, this method works really well for elemental quantification.
2. **AlignZeroLoss**: Fits a model to the zero-loss peak where the model is a Gaussian or Lorentzian function which needs to be specified by the user. This method is a lot slower and can be unstable due to its non-linear fitting procedure but has the potential to correct for subpixel shifts and works for a noisy and not sharp zero-loss peak.
3. **AlignCrossCorrelation**: Finds the shift which gives the best similarity between two spectra by cross correlation the two spectra. Subpixel accuracy can be obtained via interpolating the experimental data and finding the shift. This method is generally faster than the AlignZeroLoss but could fail if the low loss spectra are not very similar to each other. This method can also be used to align core-loss signal when no low-loss is available. 


In [5]:
fast_align = em.FastAlignZeroLoss(ll, other_spectra=[hl], cropping=True)
align = em.AlignZeroLoss(ll, other_spectra=[hl], cropping=True)
cros_align = em.AlignCrossCorrelation(ll, other_spectra=[hl], cropping=True, is_zlp=True)

In [6]:
print('Start using FastAlignZeroLoss object')
fast_align.perform_alignment()
print('Stop using FastAlignZeroLoss object')
print('Start using AlignZeroLoss object')
align.perform_alignment()
print('Stop using AlignZeroLoss object')
print('Start using AlignCrossCorrelation object')
cros_align.perform_alignment()
print('Stop using AlignCrossCorrelation object')

Start using FastAlignZeroLoss object
Stop using FastAlignZeroLoss object
Start using AlignZeroLoss object
Estimates the parameters for the fitting procedure


16384it [00:43, 375.54it/s]
16384it [00:03, 4175.17it/s]


Stop using AlignZeroLoss object
Start using AlignCrossCorrelation object


16384it [00:07, 2075.20it/s]
16384it [00:03, 4352.84it/s]

Stop using AlignCrossCorrelation object


The applied shift can be visualized

In [7]:
fig = fast_align.show_shift()

The average zero-loss peak can be shown to see if it indeed improved the sharpness. If this is not the case then something went wrong during the alignment procedure

In [8]:
fig = fast_align.show_alignment_result()

In [9]:
#redefining the new alignemet multispectra
hl_al = fast_align.aligned_others[0]
ll_al = fast_align.aligned

## Operations on Spectrum and MultiSpectrum
Some simple operations which can be performed on spectra and multispectra are shown

The raw data and energy axis can be accessed by following command

In [50]:
raw_data = hl_al[0,0].data
print('raw data')
print(raw_data)
energy_axis = hl_al.energy_axis
print('energy axis')
print(energy_axis)

raw data
[961.99895316 941.         940.99840481 ...  19.99945165   9.00004985
  10.0000997 ]
energy axis
[ 205.   205.5  206.  ... 1216.5 1217.  1217.5]


Get the aveage spectrum of the multispectrum

In [46]:
spec = hl_al.mean()

Get the index of the 315 eV energy position.

In [47]:
index = hl_al.get_energy_index(315)
print('The index of the 315 eV energy loss is: '+str(index))

The index of the 315 eV energy loss is: 220


Gaussian smooting of the spectrum, the boundaries will not resemble reality since the convolution is performed with FFT which assumes symmetric boundary conditions. 

In [52]:
smth = hl_al[10,10,:].gaussiansmooth(2)
fig, ax  = plt.subplots()
ax.plot(hl_al.energy_axis, hl_al[10,10].data, label='Raw')
ax.plot(smth.energy_axis, smth.data, label='Smoothed')
ax.legend()

The multispectrum object can be sliced to select a part of the multispectrum. Slicing is not implemented for the energy direction.

In [22]:
sub = hl_al[4:10, 3:8, :]
print('x size of the multispectrum is: '+str(sub.xsize))
print('y size of the multispectrum is: '+str(sub.ysize))
sub.mean().plot()

x size of the multispectrum is: 6
y size of the multispectrum is: 5


To get a subpart of the energy axis, following function can be used

In [27]:
sub = hl_al.get_interval((500.,900.))
print('Start of the energy axis is: '+str(sub.offset))
sub.mean().plot()

Start of the energy axis is: 500.0


Integrating the multispectrum over a certain energy range

In [43]:
integral = hl_al.integrate((600,800))
fig, ax = plt.subplots()
ax.imshow(integral)

Rebinning on the multispectrum can be performed. The rebinning factors should be positive integers since a kernel of rebinning size is used to perform the rebinning

In [37]:
rb = (4,2,2)
hl_rb = hl_al.rebin(rb)

In [38]:
print('new xsize is: ' +str(hl_rb.xsize))
print('new ysize is: ' +str(hl_rb.ysize))
print('new Esize is: ' +str(hl_rb.size))

new xsize is: 32
new ysize is: 64
new Esize is: 1013


In [39]:
hl_rb.mean().plot()

## Visualization methods
Some classes are defined which provide some live user input to navigate through the data. 

##### MultiSpectrumVisualizer
The em.MultiSpectrumVisualizer object gives the ability to navigate through a MultiSpectrum.
- The arrow on the keyboard are used to change the rectangle position,
- '+' key increases the area on which to visualize the spectrum and takes the average.
-  '-' key decreases area
-  Mouse can be used to drag rectangle over multispectrum

In [10]:
#shows one multispectrum
em.MultiSpectrumVisualizer([hl_al], labels=['Experimental data'])

In [11]:
#multiple multispectra with the same xsize, ysize can be shown.
em.MultiSpectrumVisualizer([ll_al, hl_al], labels=['low loss', 'core loss'], logscale=True)

c:\users\djannis\pycharmprojects\project\pyeelsmodel\pyEELSMODEL\operators\multispectrumvisualizer.py:265: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  ax[1].set_ylim([miny, maxy])


A line spectrum can be made from the multispectrum by averaging along one axis

In [23]:
hl_line = hl_al.mean(1)

In [24]:
em.MultiSpectrumVisualizer([hl_line])

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


##### AreaSelection
The em.AreaSelection object gives the ability to draw shapes on the map and extract the average spectrum from that area. The right mouse click is used to select the points of the area

In [12]:
area = em.AreaSelection(hl_al, max_points=5, other_spectra=[ll_al])

The .determine_input_area() shows the image on which to select a region.

In [13]:
area.determine_input_area()

the shape is drawn, this will be used to determine the average spectrum


In [15]:
#shows the drawn area
area.show_area()

In [14]:
spec_mean =  area.get_mean_from_area()

In [16]:
#the average spectrum of the added multispectra
ll_mean = area.other_avg_spec[0]

In [17]:
spec_mean.plot()

In [18]:
ll_mean.plot()

## Background Removal
In this part a small example is shown on how to remove the background from a multispectrum. The BackgroundRemoval class has a workflow implemented on how to get the appropriate results. Multiple methods of background subtraction are implemented but in this case we focus on the power-law only.

In [54]:
em.BackgroundRemoval?

Init signature:
em.BackgroundRemoval(
    spectrum,
    signal_range,
    model_type='Powerlaw',
    linear_fitting=False,
    order=2,
    non_linear_fitter='LSQ',
    r_values=(2, 3),
)
Docstring:     
BackgroundRemoval is a class which needs a spectrum and signal range. Using this type of
information it can remove the background eels signal. Different types of background models
are implemented.
Parameters
----------
spectrum: Spectrum
    The spectrum from which the background should be removed
signal_range: tuple
    Indicates the region on which the background fitting should be performed.
    This object can perform the fitting on two area and then two tuples should be
    given.
model_type: str
    Indicates which model is used to perform the background fitting. At this point
    Powerlaw, FastBG, Exponential en Polynomial is implemented. (default: Powerlaw)
linear_fitting: bool
    Indicates if a linear fitting procedure is used to determine the background. This
    can only wor

In [61]:
back = em.BackgroundRemoval(hl_al, (600,700)) #before the iron L edge

In [62]:
rem = back.calculate_multi() 0#rem is the background removed spectrum 

True


16384it [00:02, 6425.89it/s]
16384it [00:50, 323.21it/s]
16384it [00:01, 14190.95it/s]


In [63]:
fig = back.show_fit_result(use_mean=True)

In [65]:
em.MultiSpectrumVisualizer([hl_al, rem, back.multi_model_signal], labels=['Raw spectrum', 'Background subtracted', 'Background'])

## Model-based Quantification
The last part shows how to get the elemental quantification on the multispectrum. It uses the ElementalQuantification class which has a workflow defined which only needs information on the estimated elements, acceleration voltage, convergence angle and collection angle. **See other notebooks for more in-depth examples on how to perform model-based quantification.**

In [68]:
elements = ['C', 'N', 'O', 'Fe']
edges = ['K', 'K', 'K', 'L']
E0 = 300e3 
alpha = 1e-9
beta = 20e-3 
settings = (E0, alpha, beta)

In [69]:
quant = em.ElementalQuantification(hl, elements, edges, settings, ll=ll)
quant.n_bgterms = 4
quant.linear_fitter_method = 'ols'
quant.do_procedure()

cannot use analytical gradients since a convolutor is inside model
16384it [00:39, 415.54it/s]
16384it [00:11, 1402.04it/s]


In [70]:
#calculate different models with different components added to it. 
multimodels = quant.get_multimodels()

16384it [00:12, 1292.75it/s]
16384it [00:12, 1264.49it/s]
16384it [00:12, 1331.54it/s]
16384it [00:12, 1351.66it/s]
16384it [00:11, 1373.19it/s]


In [73]:
#compare the fitted model to experimental data
em.MultiSpectrumVisualizer([quant.spectrum, multimodels[-1]])

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [74]:
quant.show_elements_maps()